In [1]:
#pip install pyserial
import serial, time
class KA3005P():
    def __init__(self): 
        self.serial = serial.Serial(port = '/dev/ttyACM0', 
                   baudrate = 9600, 
                   parity   = serial.PARITY_NONE,
                   bytesize = serial.EIGHTBITS, 
                   stopbits = serial.STOPBITS_ONE, 
                   xonxoff=False, rtscts=False, dsrdtr=False )
        self.delay = 0.05 # You need to give UART comm a rest
        self.STATUS()
        return None
        
    @property
    def dI(self):
        time.sleep(self.delay)
        self.serial.write(b'ISET1?') 
        return float(self.serial.read(5).decode('utf8'))

    @dI.setter
    def dI(self, value):
        time.sleep(self.delay)
        if value < 5.1 and value > 0: 
            self.serial.write(f'ISET1:{value:.3f}'.encode('utf8'))
            print(f'Desired current was set to be {value:.3f}A') 
        else:
            print('Input a current value in (0A, 5.1A)')
        return None
        
    @property
    def dV(self):  
        time.sleep(self.delay)
        self.serial.write(b'VSET1?') 
        return float(self.serial.read(5).decode('utf8'))

    @dV.setter
    def dV(self, value):
        time.sleep(self.delay)
        if value < 31 and value > 0: 
            self.serial.write(f'VSET1:{value:.2f}'.encode('utf8'))   
            print(f'Desired voltage was set to be {value:.2f}V')
        else:
            print('Input a voltage value in (0A, 31A)')
        return None
    
    @property    
    def rI(self):
        time.sleep(self.delay)
        self.serial.write(b'IOUT1?') 
        return float(self.serial.read(5).decode('utf8'))
    
    @property
    def rV(self):
        time.sleep(self.delay)
        self.serial.write(b'VOUT1?')
        return float(self.serial.read(5).decode('utf8')) 
    
    @property
    def BEEP(self):
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)  
        return {0:'OFF', 1:'ON'}[ord(stat) >> 4 & 0b1]
        
    @BEEP.setter
    def BEEP(self, value):
        time.sleep(self.delay)
        if value == 'ON':
            self.serial.write(b'BEEP1')  
        elif value == 'OFF':
            self.serial.write(b'BEEP0')  
        else:
            print("You either use 'ON' to enable the Buzzer or 'OFF' to disable it")
        return None
    
    @property
    def OUT(self): 
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)
        return {0:'OFF', 1:'ON'}[ord(stat) >> 6 & 0b1]

    @OUT.setter
    def OUT(self, value):
        time.sleep(self.delay)
        if value == 'ON':
            self.serial.write(b'OUT1')
        elif value == 'OFF':
            self.serial.write(b'OUT0')  
        else:
            print("You either use 'ON' to enable the power supply or 'OFF' to disable it")
        return None
    
    def STATUS(self): 
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)
        print( ', '.join(v[ord(stat) >> b & 0b1] for b, v in {
            0 : {0:'Constant current mode', 1:'Constant voltage mode'},
            4 : {0:'BEEP.OFF' , 1:'BEEP.ON' },
            5 : {0:'OCP.OFF'  , 1:'OCP.ON'  }, # Overcurrent Protection
            6 : {0:'OUT.OFF'  , 1:'OUT.ON'  },
            7 : {0:'OVP.OFF'  , 1:'OVP.ON'  }  # Overvoltage Protection
        }.items()) )
        return None
    
    @property
    def MODE(self): 
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)  
        return {0:'Constant current mode', 1:'Constant voltage mode'}[ord(stat) >> 0 & 0b1]
        
    @property
    def IDN(self):
        time.sleep(self.delay)
        self.serial.write(b'*IDN?') 
        return self.serial.read(30).decode('utf8')
     
    @property
    def RCL(self):
        return None
    
    @RCL.setter
    def RCL(self, value):
        time.sleep(self.delay)
        if value < 6 and value > 0: 
            self.serial.write(f'RCL{value:d}'.encode('utf8'))   
            print(f'Settings stored in M{value:d} was loaded')
        else:
            print('Choose a memory slot value in [1, 5]')
        return None
    
    @property
    def SAV(self):
        return None
    
    @SAV.setter
    def SAV(self, value):
        time.sleep(self.delay)
        if value < 6 and value > 0: 
            self.serial.write(f'SAV{value:d}'.encode('utf8'))  
            print(f'Current settings was stored in M{value:d}')
        else:
            print('Choose a memory slot value in [1, 5]')
        return None

    @property
    def OCP(self):
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)
        return {0:'OFF', 1:'ON'}[ord(stat) >> 5 & 0b1] 
    
    @OCP.setter
    def OCP(self, value):
        time.sleep(self.delay)
        if value == 'ON':
            self.serial.write(b'OCP1')  
        elif value == 'OFF':
            self.serial.write(b'OCP0')  
        else:
            print("You either use 'ON' to enable the Overcurrent Protection or 'OFF' to disable it")
        return None
    
    @property
    def OVP(self):
        time.sleep(self.delay)
        self.serial.write(b'STATUS?')
        stat = self.serial.read(1)  
        return {0:'OFF', 1:'ON'}[ord(stat) >> 7 & 0b1]
        
    @OVP.setter
    def OVP(self, value):
        time.sleep(self.delay)
        if value == 'ON':
            self.serial.write(b'OVP1')  
        elif value == 'OFF':
            self.serial.write(b'OVP0')  
        else:
            print("You either use 'ON' to enable the Overvoltage Protection or 'OFF' to disable it")
        return None
    
    @property
    def LOCK(self):
        return None
    
    @LOCK.setter
    def LOCK(self, value): 
        time.sleep(self.delay)
        if value == 'ON':
            self.serial.write(b'LOCK1')  
        elif value == 'OFF':
            self.serial.write(b'LOCK0')  
        else:
            print("You either use 'ON' to enable the lock or 'OFF' to disable it")
        return None 
    
    def close(self):
        self.serial.close()

In [2]:
ka = KA3005P()

Constant current mode, BEEP.ON, OCP.ON, OUT.OFF, OVP.ON


### Give KA3005P a initial settings

In [4]:
ka.dI, ka.dV = 2.3, 1.3 

Desired current was set to be 2.300A
Desired voltage was set to be 1.30V


### See the initial settings

In [5]:
ka.dI, ka.dV

(2.3, 1.3)

### Change the settings

In [6]:
ka.dI, ka.dV = 4.3, 4.3 

Desired current was set to be 4.300A
Desired voltage was set to be 4.30V


### See the new settings

In [7]:
ka.dI, ka.dV

(4.3, 4.3)

### See if the output is enabled

In [8]:
ka.OUT 

'OFF'

### Enable the Output

In [9]:
ka.OUT = 'ON'

In [ ]:
ka.OUT = 'OFF'

### See if the output is enabled

In [10]:
ka.OUT 

'ON'

### See the real output value

In [11]:
ka.rI, ka.rV  

(0.0, 4.3)

### Check Buzzer

In [12]:
ka.BEEP

'ON'

### Disable Buzzer

In [13]:
ka.BEEP = 'OFF'

### See the change

In [14]:
ka.BEEP

'OFF'

### See the info of the machine

In [15]:
ka.IDN

'KORAD KA3005P V5.8 SN:03384597'

### Switch to the specified memory slot

In [17]:
ka.RCL = 3

Settings stored in M3 was loaded


In [18]:
ka.dV, ka.dI

(3.0, 3.0)

### Change and Save it the memory slot

In [19]:
ka.dV, ka.dI = 4, 4

Desired voltage was set to be 4.00V
Desired current was set to be 4.000A


In [20]:
ka.SAV = 3

Current settings was stored in M3


In [21]:
ka.RCL = 1
time.sleep(3)
ka.RCL = 3

Settings stored in M1 was loaded
Settings stored in M3 was loaded


In [22]:
ka.dV, ka.dI

(4.0, 4.0)

### Enable Ovecurrent an Overvoltage Protection

In [ ]:
ka.OCP, ka.OVP = 'ON', 'ON'

### Lock the control Panel

In [24]:
ka.LOCK = 'ON'